# Crawl data from Google Images

## Make the dataset connected into Azure

In [ ]:
# Install required packages
# %pip install beautifulsoup4
# %pip install openpyxl
# !pip install selenium pillow
# !pip install webdriver_manager

In [ ]:
%restart_python

In [ ]:
# Import necessary packages
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [ ]:
# Load Excel data
ethnic_df = pd.read_excel("Datasets/Ethnic_Origin_Facial_Issues.xlsx")
issue_df = pd.read_excel("Datasets/Frequent_Facial_Issues.xlsx")

In [ ]:
# Add a constant key and perform a cross join
ethnic_df['key'] = 1
issue_df['key'] = 1
cross_joined_df = pd.merge(ethnic_df, issue_df, on='key').drop('key', axis=1)

In [ ]:
# Check the first few rows
cross_joined_df.head()

In [ ]:
# Check the shape of the DataFrame
cross_joined_df.shape

In [ ]:
# Extract column names
col_name = list(cross_joined_df.columns)
col_name

In [ ]:
# Create a new DataFrame with selected columns
df = cross_joined_df[['Ethnic Origin', 'Issue']]
df.head()

In [ ]:
# Add a new column for search keywords
df['key_word'] = df['Ethnic Origin'] + "'s " + df['Issue']
df.head()

In [ ]:
# Iterate over rows to check keywords
for index, row in df.iterrows():
    print(row['key_word'])

In [ ]:
# Set up directory for downloading images
import os
os.mkdir("/Workspace/Users/hanchen@845861076.onmicrosoft.com/Dermatology_LLM/images")
os.chdir("/Workspace/Users/hanchen@845861076.onmicrosoft.com/Dermatology_LLM/images")
print(os.getcwd())

# Crawl images from Google
for index, row in df.iterrows():
    search_query = row['key_word']
    url = f"https://www.google.com/search?hl=en&tbm=isch&q={search_query}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    images = soup.find_all('img')
    image_urls = [img['src'] for img in images if img['src'].startswith('http')]

    # Create a folder for each issue
    if not os.path.exists(row['Issue']):
        os.makedirs(search_query)

    # Download first 10 images
    count = 0
    for img_url in image_urls:
        if count >= 10:
            break
        try:
            img_data = requests.get(img_url).content
            img_size = len(img_data)
            if img_size > 50000:
                with open(f"{search_query}/image_{count+1}.jpg", 'wb') as f:
                    f.write(img_data)
                print(f"Downloaded {search_query}/image_{count+1}.jpg")
                count += 1
        except Exception as e:
            print(f"Failed to download image: {e}")